<a href="https://colab.research.google.com/github/shown5/HandsOnLLM/blob/main/chap3_diveIntoLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers>=4.41.2 accelerate>=0.31.0

In [ ]:
# import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# モデルとトークナイザの読み込み
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)

# パイプラインの作成
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# ガーデニングで起きてしまった悲しい出来事についてサラに謝罪するメールを書いてください
# どのようにして起こったのかもか説明してください。
prompt = "Write an email appologizing to Saarah for the tragic gardening mishap. Explain how it happened."

# Add the attention_mask to the generator call
output = generator(prompt, use_cache=False)

print(output[0]["generated_text"])

In [ ]:
# プロンプト：フランスの首都は
prompt = "The capital of France is"

# 入力プロンプトをトークン化
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

# lm_headの前のモデルの出力を取得
model_output = model.model(input_ids)

# lm_headの出力を取得
lm_head_output = model.lm_head(model_output[0])

In [ ]:
token_id = lm_head_output[0, -1].argmax(-1)
tokenizer.decode(token_id)

In [ ]:
model_output[0].shape

In [ ]:
lm_head_output.shape

In [ ]:
prompt = "Write a very long email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

# 入力プロンプトをトークン化
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

In [ ]:
%%timeit -n 1
# テキストを生成
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    use_cache=True
)

In [ ]:
%%timeit -n 1
# テキストを生成
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    use_cache=False
)